<a href="https://colab.research.google.com/github/SYEON9/natural_language_3th/blob/main/HW/%5BHW13%5DPolynomial_Regression_%26_Regularization_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#linear regression은 데이터를 가장 잘 설명해주는 직선을 찾는 것이었다.
#그러나 데이터의 형태가 곡선의 형태라면 어떻게 예측해야할까?
#이 문제를 우리는 polymomial regression을 사용해서 알아볼 것이다. 
#그리고 regularization을 통해 어떻게 학습이 변화되는지 알아보자. 

A Special case of multiple linear regression

In [ ]:
#4차 polynomial 함수를 이용하여 데이터를 생성해보자. 
from matplotlib import pyplot
from autograd import grad 
from autograd import numpy 

In [ ]:
numpy.random.seed(0)
x = numpy.linspace(-3, 3, 20)
y = x**4 + x**3 - 4*x**2 + 8*numpy.random.normal(size=len(x))
pyplot.scatter(x,y);

In [ ]:
#데이터가 직선형태가 아니므로 linear regression 대신 다른 방법을 사용해야 한다. 
#우리는 polynomial function을 사용하여 데이터를 표현할 것이다. 

#1. d차 곡선을 정의하자
#우리가 찾아야 할 것은 w(가중치)이다.  이 문제를 우리는 행렬의 선형결합 형태로 나타낼 수 있다.
#이것을 고려하여 실습해보자.

#3차항까지 고려하겠다.
degree = 3

#행렬의 행태로 x값 변환
def polynomial_features(x, degree):
    """ Generate polynomial features for x."""
    
    X = numpy.empty((len(x), degree+1))        #배열 생성. zeros와 달리 메모리를 할당발다 초기화 없이 반환하여 해당 메모리에 남아있는 값이 들어간다. 
    for i in range(degree+1):
        X[:,i] = x**i                          #첫번째 열에는 1을, 나머지 열에는 x의 i제곱을 각각 넣는다. 
    return X

X = polynomial_features(x, degree)
print(X.shape)



In [ ]:
X

Scale the data, train the model

In [ ]:
#input vairable을 normalize하고 학습을 진행하자.
#scikit-learn의 min max scaler를 사용한다.

from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)         #X[:,0]의 값이 0으로 변함.
print(X_scaled)
X_scaled[:,0] = 1

In [ ]:
X_scaled.shape

In [ ]:
#shape을 다 다듬었으니 이제 학습을 진행해보자.
def linear_regression(params, X):
    '''The linear regression model in matrix form. 
    Arguments:
      params: 1D array of weights for the linear model
      X     : 2D array of input values
    Returns:
      1D array of predicted values
    '''
    return numpy.dot(X, params)                           #numpy.dot(): array를 곱함

def mse_loss(params, model, X, y):
    '''The mean squared error loss function.
    Arguments:
      params: 1D array of weights for the linear model
      model : function for the linear regression model
      X     : 2D array of input values
      y     : 1D array of predicted values
    Returns:
      float, mean squared error
    '''
    y_pred = model(params, X)
    return numpy.mean(numpy.sum((y-y_pred)**2))

gradient = grad(mse_loss)

In [ ]:
max_iter = 3000
alpha = 0.01                               #learning rate
params = numpy.zeros(X_scaled.shape[1])
descent = numpy.ones(X_scaled.shape[1])
i = 0

In [ ]:
from sklearn.metrics import mean_absolute_error

while numpy.linalg.norm(descent) > 0.01 and i < max_iter:            #loss의 평균값(표준값)이 0.01보다 작거나 max_iter를 넘기면 멈춘다. 
    descent  = gradient(params, linear_regression, X_scaled, y)
    params = params - descent*alpha
    loss = mse_loss(params, linear_regression, X_scaled, y)
    mae = mean_absolute_error(y, X_scaled@params)
    if i%100 == 0:
        print("iteration {}, loss = {}, mae = {}".format(i, loss, mae))
    i += 1

In [ ]:
#학습된 params확인
params

In [ ]:
#원래 데이터와 함께 그래프로 그려보자.
xgrid = numpy.linspace(x.min(), x.max(), 30)              #linspace: 1차원 벡터 생성.
Xgrid_poly_feat = polynomial_features(xgrid, degree)      #데이터 왼쪽에 1벡터를 추가
Xgrid_scaled = min_max_scaler.transform(Xgrid_poly_feat)  #추가한 벡터가 0으로 바뀐다.
Xgrid_scaled[:,0] = 1                                     #다시 1로 바꿔준다. 

#시각화
pyplot.scatter(x, y, c = 'r', label = 'true')
pyplot.plot(xgrid, Xgrid_scaled@params, label = 'predicted')  #xgrid로 범위 지정. 
pyplot.legend();

Observe Underfitting & Overfitting


In [ ]:
#1~15까지의 다양한 차수를 가지고 학습을 진행해보자. 
#이 과정에서 어떤 변화가 찾아올까?
from urllib.request import urlretrieve
URL = 'https://raw.githubusercontent.com/engineersCode/EngComp6_deeplearning/master/scripts/plot_helpers.py'
urlretrieve(URL, 'plot_helpers.py')

In [ ]:
#먼저 데이터를 확인하자
from plot_helpers import interact_polyreg

max_degree = 15
interact_polyreg(max_degree, x, y)            #차수에 따른 polynomial regreesion을 살펴보자. 

Underfitting

In [ ]:
#underfitting하면 우리는 high bias, low variance를 갖게된다. 
#즉, 예측값을 규칙있게 뽑아내지만(예측값이 한군데에 몰려있지만) 정답과 거리가 있다.


Overfitting

In [ ]:
#degree를 높게 설정하면 w의 개수가 증가-> 학습 데이터에 대해 잘 예측하고 mae도 감소
#그러나 새로운 데이터를 잘 못맞춤. 
#high variance, low bias이다. 


Regularization

In [ ]:
#overfitting을 막는 방법. 
#cost function에 새로운 식을 추가하여 복잡한 모델이 되는 것을 막는다. 
#regularization을 추가하면 loss를 줄이기 위해 가중치는 작은 값을 선호하게 된다. 
#여기서 람다는 얼마나 큰 제약을 줄것인가를 결정하고 큰 값을 가질수록 가중치를 더 작게 만든다.


In [ ]:
#람다를 1로 설정하여 실습을 해보자
def regularized_loss(params, model, X, y, _lambda = 1.0):
    '''The mean squared error loss function with an L2 penalty.
    Arguments:
      params: 1D array of weights for the linear model
      model : function for the linear regression model
      X     : 2D array of input values
      y     : 1D array of predicted values
      _lambda: regularization parameter, default 1.0
    Returns:
      float, regularized mean squared error
    '''
    y_pred = model(params, X)
    return numpy.mean( numpy.sum((y-y_pred)**2)) + _lambda * numpy.sum(params[1:]**2)

gradient = grad(regularized_loss)

In [ ]:
no_regularization_params = params.copy()

In [ ]:
max_iter = 3000
alpha = 0.01
params = numpy.zeros(X_scaled.shape[1])
descent = numpy.ones(X_scaled.shape[1])      #cost의 평균을 담을 것.
i = 0

#gradient descent
from sklearn.metrics import mean_absolute_error

while numpy.linalg.norm(descent) > 0.01 and i < max_iter:
    descent = gradient(params, linear_regression, X_scaled, y)
    params = params - descent*alpha
    loss = mse_loss(params, linear_regression, X_scaled, y)
    mae = mean_absolute_error(y, X_scaled@params)
    if i%100 == 0:
        print('iteration {}, loss = {}, mae = {}'.format(i, loss, mae))
    i += 1

In [ ]:
#regularization을 추가하지 않은 것과 비교해보자.
print('weights without regularization')
print(no_regularization_params)
print('weights with regularization')
print(params)
pyplot.scatter(x, y, c = 'r', label = 'true')
pyplot.plot(xgrid, Xgrid_scaled@no_regularization_params, label = 'w/0 regularization')
pyplot.plot(xgrid, Xgrid_scaled@params, label = 'with regularization')
pyplot.legend();

In [ ]:
#더 다양한 차우에서 regularization을 비교해보자.
interact_polyreg(max_degree, x, y, regularized = True)

In [ ]:
#degree가 커질수록 regularization의 중요성이 들어난다. 
#regularization을 사용하면 높은 차수에서 예측 모델이 부드럽게 학습 데이터들을 지나간다. 
#그러므로 모델의 예측 성능도 개선된다. 